# Métodos predictivos: tarea de asignación (semana 4)
## Implementación propia del método de vecinos más cercanos. Comparación con otros métodos en *scikit-learn*.

## Instrucciones
En este notebook encontrarás los pasos necesarios para realizar la tarea de la 4ª semana de Métodos Predictivos del Máster en Ciencia de Datos.Lea detenidamente y siga los pasos indicados en las siguientes celdas, complete el código donde se indique ``# COMPLETAR AQUI``, respetando el formato o los nombres de funciones especificados.

## Descripción de la tarea
En esta tarea, el principal objetivo será realizar una implementación propia del método de k vecinos más cercanos (kNN), analizar su comportamiento mediante un estudio de sus parámetros, y por último compararlo con otros métodos de scikit-learn. La tarea consta de varios apartados:
0. Carga y preparación de datos
1. Implementar función de distancia (1 punto)
2. Implementar kNN básico (2 puntos)
3. Implementar kNN con pesos (1 punto)
4. Estudio de los parámetros (0.75 puntos)
5. Comparación con otros métodos (0.25 puntos)



Rellenar esta celda con los datos del alumno

**Nombre**: Juan José

**Apellidos**: Méndez Torrero

## 0. Carga y preparación de datos

En primer lugar, vamos a cargar los datos que utilizaremos posteriormente para entrenar los distintos métodos de clasificación, incluyendo nuestra implementación de kNN.

Vamos a utilizar un subconjunto del *dataset* [*Mice Protein Expression*](https://www.kaggle.com/ruslankl/mice-protein-expression). La versión de los datos que vamos a utilizar se encuentran en el siguiente [enlace](http://www.uco.es/users/jmoyano/MiceProteinExpression.csv). 

El conjunto de datos contiene los niveles de expresión de 77 proteínas medidas en el cortex cerebral de 8 tipos de ratones.
Se realizaron hasta 15 medidas por ratón, incluyendo 38 ratones de control, y 34 trisómicos (síndrome de down); es decir, un total de 72 ratones. Por tanto, hay un total de 570 muestras (38x15) para ratones de control, y 510 para trisómicos (34x15). En total, el dataset contiene 1080 muestras, que pueden considerarse independientes.

Existen 8 clases distintas, cada una descrita por 3 valores x-Y-z, relativos a genotipo, comportamiento, y tratamiento de los ratones:
  - x: c (control) / t (trisomia)
  - Y: CS (estimulados a aprender) / SC (no recibe estimulación)
  - z: s (se le inyecta *saline*) / m (se le inyecta *memantine*) 

Para más información acerca de los datos puede consultar la [fuente](https://www.kaggle.com/ruslankl/mice-protein-expression).

En la siguiente celda, cargaremos los datos directamente utilizando una [url](http://www.uco.es/users/jmoyano/MiceProteinExpression.csv) donde estén alojados, o descargandolos y añadiendolos a nuestro espacio de trabajo en Google Colab.

In [1]:
import pandas as pd
import numpy as np

# Leemos los datos desde una URL externa
data = pd.read_csv('http://www.uco.es/users/jmoyano/MiceProteinExpression_211018.csv')

# Tras cargarlos, imprimimos para ver algunas de las columnas y sus valores
print(data)

# Imputar posibles valores perdidos con la media de la columna
# Este proceso formaria parte del pre-procesado de datos;
#   realizado simplemente para poder trabajar con ellos, sin entrar en más detalle
from sklearn.impute import SimpleImputer
for column in data.columns:
  if data[column].isnull().values.any():
    data[[column]] = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(data[[column]])

       MouseID  DYRK1A_N   ITSN1_N    BDNF_N     NR1_N    NR2A_N    pAKT_N  \
0        309_1  0.503644  0.747193  0.430175  2.816329  5.990152  0.218830   
1        309_2  0.514617  0.689064  0.411770  2.789514  5.685038  0.211636   
2        309_3  0.509183  0.730247  0.418309  2.687201  5.622059  0.209011   
3        309_4  0.442107  0.617076  0.358626  2.466947  4.979503  0.222886   
4        309_5  0.434940  0.617430  0.358802  2.365785  4.718679  0.213106   
...        ...       ...       ...       ...       ...       ...       ...   
1075  J3295_11  0.254860  0.463591  0.254860  2.092082  2.600035  0.211736   
1076  J3295_12  0.272198  0.474163  0.251638  2.161390  2.801492  0.251274   
1077  J3295_13  0.228700  0.395179  0.234118  1.733184  2.220852  0.220665   
1078  J3295_14  0.221242  0.412894  0.243974  1.876347  2.384088  0.208897   
1079  J3295_15  0.302626  0.461059  0.256564  2.092790  2.594348  0.251001   

       pBRAF_N  pCAMKII_N   pCREB_N  ...     BAD_N  BCL2_N     

In [2]:
# Vamos a conocer ciertas características de nuestros datos

# Imprimimos información del conjunto de datos al completo
print('Información del dataset al completo')
print(data.info())
print('---\n')

# Número de columnas
print('Columnas: ' + str(len(data.columns)))

# Número de filas
print('Filas: ' + str(len(data)))

# Almacenar e imprimir los distintos valores de las clases
clases = data['class'].unique()
print('Clases: ' + str(clases))

# Apariciones de cada clase
print(data['class'].value_counts())

Información del dataset al completo
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 79 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MouseID          1080 non-null   object 
 1   DYRK1A_N         1080 non-null   float64
 2   ITSN1_N          1080 non-null   float64
 3   BDNF_N           1080 non-null   float64
 4   NR1_N            1080 non-null   float64
 5   NR2A_N           1080 non-null   float64
 6   pAKT_N           1080 non-null   float64
 7   pBRAF_N          1080 non-null   float64
 8   pCAMKII_N        1080 non-null   float64
 9   pCREB_N          1080 non-null   float64
 10  pELK_N           1080 non-null   float64
 11  pERK_N           1080 non-null   float64
 12  pJNK_N           1080 non-null   float64
 13  PKCA_N           1080 non-null   float64
 14  pMEK_N           1080 non-null   float64
 15  pNR1_N           1080 non-null   float64
 16  pNR2A_N          1080 no

Una vez visualizadas las características de nuestro conjunto de datos, podemos observar ciertas características:

*   Contiene una primera columna de ID
*   Contiene atributos unicamente de tipo numérico
*   La última columna contiene el atributo de clase
*   Tenemos 8 clases distintas, bastante balanceadas. La más frecuente aparece en 150 patrones, y la menos en 105.

Por tanto, deberíamos hacer, al menos, el siguiente pre-procesado de los datos:
*   Eliminar la columna ID
*   Separar los atributos de entrada y la clase en variables distintas

Como se indica, en casos en que sea necesario, puede filtrar columnas de los datos para que las ejecuciones sean menos costosas (vea comentario en el código). No es necesario realizar dicha selección de variables, simplemente en caso que el alumno considere para tener ejecuciones más rápidas.

Además, vamos a realizar un particionado de los datos para su posterior utilización. En este caso, vamos a realizar una partición de los datos aleatoriamente en hold-out, utilizando un 70% de los datos para entrenamiento y el 30% restante para test. Esas particiones se usarán en adelante para todos los métodos, para así obtener unos resultados consistentes.


In [3]:
# Atributos de entrada en X (eliminamos ID y clase)
X = data.drop(columns=['MouseID', 'class'])

# El proceso de predicción de kNN puede ser costoso.
# Si el estudiante lo considera, puede reducir el número de atributos del conjunto 
#   de datos como se indica en las siguientes líneas. En dichas líneas se están 
#   manteniendo las n primeras columnas de los datos.
# Los resultados obviamente cambiarán dependiendo del número de atributos que mantengamos
#   en el conjunto de datos (por lo general, al reducir el número de atributos
#   obtendremos peores resultados).
# Sin embargo, se espera que el código funcione igualmente (aunque más lento) para
#   el conjunto de datos completo, y que las respuestas a las distintas preguntas
#   sea consistente con los resultados obtenidos con las n primeras columnas que
#   se mantengan.

# n = 30
# X = X[X.columns[0:n]]

# Atributo de clase en y
y = data['class']

# Particionado de los datos en entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Por último, vamos a realizar un escalado de los datos, de modo que se encuentren en el rango [0, 1].
A partir de ahora, vamos a considerar las particiones de entrenamiento y test que ya tenemos, por lo que la función de escalado la "entrenaremos" utilizando la partición de entrenamiento, y posteriormente la aplicaremos a ambos subconjuntos.


In [4]:
# Ignoramos warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Escalamos los atributos al rango [0, 1]; unicamente columnas numéricas (float o int)
from sklearn.preprocessing import MinMaxScaler
for column in X_train.columns:
  if 'float' in str(X_train.dtypes[column]) or 'int' in str(X_train.dtypes[column]):
    scaler = MinMaxScaler()
    X_train[[column]] = scaler.fit_transform(X_train[[column]])
    X_test[[column]] = scaler.transform(X_test[[column]])

# Imprimimos dataset al completo, y valores minimo y maximo de cada columna, para comprobar que se realizó correctamente
# En entrenamiento, los valores mínimos deben ser 0 y los máximos 1 en todas las variables
print(X_train)
print(X_train.min(axis=0))
print(X_train.max(axis=0))

# En test, los valores mínimo y máximo no deben ser estrictamente 0 y 1 respectivamente
#   pero se les espera cercanos a dichos valores.
print(X_test)
print(X_test.min(axis=0))
print(X_test.max(axis=0))

      DYRK1A_N   ITSN1_N    BDNF_N     NR1_N    NR2A_N    pAKT_N   pBRAF_N  \
271   0.054181  0.106474  0.471912  0.517374  0.367249  0.411274  0.516004   
140   0.199776  0.258534  0.618210  0.475502  0.419707  0.363898  0.480685   
649   0.268964  0.337061  0.618420  0.521633  0.432118  0.351904  0.428993   
194   0.003612  0.015133  0.113809  0.147046  0.143260  0.258068  0.359064   
367   0.660648  0.613879  0.501000  0.460337  0.386189  0.325288  0.452642   
...        ...       ...       ...       ...       ...       ...       ...   
1033  0.052823  0.059351  0.553245  0.296299  0.150065  0.442981  0.524046   
763   0.080165  0.145415  0.608576  0.464178  0.307266  0.531871  0.653196   
835   0.096514  0.136877  0.299511  0.256445  0.250052  0.328984  0.411487   
559   0.033625  0.049020  0.513055  0.442781  0.447294  0.363979  0.388182   
684   0.105776  0.133157  0.468060  0.368817  0.286204  0.352954  0.480359   

      pCAMKII_N   pCREB_N    pELK_N  ...     SHH_N     BAD_N   

## 1. Implementar función de distancia (1 punto)

Dado que kNN es un método basado en vecindad, es decir, en aquellos patrones más cercanos, tendremos que definir una función, o funciones de distancia entre patrones para poder determinar su cercanía. 
**Nota:** Las funciones implementadas a lo largo del notebook tienen que funcionar no solo para este conjunto de datos, sino para cualquiera que contenga atributos reales y categóricos.

A continuación, se incluye una celda de código, donde debe definir, al menos, la siguiente función:

*   ``distancia(A, B, p)``: Esta función debe calcular la distancia entre dos patrones ``A`` y ``B`` (basándose en la [distancia de Minkowski](https://es.hrvwiki.net/wiki/Minkowski_distance), que es una generalización de otras distancias). La función devolverá un número real indicando la distancia entre ambos patrones. Más abajo se incluye la ecuación para calcular la distancia entre ``A`` y ``B``, dado el valor de ``p``. Dependiendo del valor de ``p``, la función de distancia será distinta:
  *   p=1: Equivale a la [distancia de Manhattan](https://es.wikipedia.org/wiki/Geometr%C3%ADa_del_taxista)
  *   p=2: Equivale a la [distancia Euclídea](https://es.wikipedia.org/wiki/Distancia_euclidiana)
  *   ...

$$distancia(A, B, p) = \sqrt[p]{\sum_{i=1}^{n}{\left | A_i - B_i\right | ^ p}}$$

Al calcular la distancia, tendrá que tener en cuenta que si ambos valores son valores categóricos, la diferencia entre ellos será 0 si coinciden, y 1 en caso contrario. En caso de ser valores numéricos, la diferencia es el valor absoluto de la resta entre ambos valores.

Nótese que tanto en la siguiente celda como en el resto del notebook, debe(n) existir la(s) funcion(es) requerida(s), pero se pueden implementar otras funciones auxiliares si se considera necesario sin ningún problema.


In [5]:
# En esta celda, implemente la función para calcular la distancia entre dos patrones. 
# Respete la declaración de la cabecera de la función. Si lo considera necesario, 
#   puede añadir nuevos parámetros a la función, siempre que tengan valor por defecto.

def distancia(A, B, p=2):
  '''
  Calcula la distancia con p-norma entre dos patrones A y B.
  Válida para patrones que contengan atributos tanto numéricos como categóricos

  :param A: Lista con valores para todos los atributos del patrón A
  :param B: Lista con valores para todos los atributos del patrón B
  :param p: Valor de la norma para el cálculo de distancia. Por defecto, p=2, es decir, calcula distancia euclídea
   
  :return: Valor numérico con la distancia entre ambos patrones
  '''
  ###
  # COMPLETAR AQUI
  ###

  distances = []

  if len(A) != len(B):
    return "Error. La longitud de las listas no son iguales"
  
  for i in range(0, len(A)):
    
    if type(A[i]) != type(B[i]):
      return "Error. El tipo de los parámetros A y B no coinciden"

    # Caso valor categórico
    if isinstance(A[i], str) and isinstance(B[i], str):    

        if A[i] == B[i]:
          distances.append(0)

        else:
          distances.append(1)

    # Caso valor numerico
    else:

      dis = np.power(np.abs(A[i] - B[i]), p)

      distances.append(dis)

  summatory = np.array(distances).sum()

  return np.power(summatory, 1/p)


In [6]:
# En esta celda, podrá probar si su función de distancia funciona correctamente.

# Esta distancia debería ser 0.5
print(distancia([0.8, 0.1, 0.45, 0.9], [0.7, 0.4, 0.45, 0.8], p=1))

# Esta distancia debería ser 1.0488 (redondeando)
print(distancia([0.8, 0.1, 0.45, 'A', 'b'], [0.7, 0.4, 0.45, 'A', 'c'], p=2))

# Probar que funciona con dos patrones de los datos
print(distancia(X_train.iloc[0], X_train.iloc[1]))

0.5000000000000001
1.0488088481701516
1.2222443459182537


## 2. Implementar kNN básico (2 puntos)

Una vez tenemos implementada nuestra función de distancia, podemos implementar nuestra primera versión del método kNN. En este caso, se busca implementar el método básico, donde la predicción se produce en base a los k vecinos más cercanos, teniendo todos ellos el mismo peso en la predicción. Para cada patrón de *test*, el método ha de buscar los k vecinos más cercanos en los patrones de entrenamiento, y utilizar las clases asociadas a dichos vecinos para generar su salida.

Sin embargo, se pretende que el método no devuelva una clase predicha, sino una probabilidad de pertenencia a cada clase. La probabilidad de pertenencia se calculará como el ratio de vecinos más cercanos pertenenciendo a cada clase de entre el total de vecinos. 
Como ejemplo, si estamos utilizando k=3, y hay 1 vecino de la clase A, 2 vecinos de la clase B, y 0 vecinos de la clase C, la salida debe ser una lista tal como: ``[0.333, 0.667, 0]``.

En las siguiente celda encontrará la definición de la función ``kNN`` a completar. La función debe recibir varios parámetros:
*   ``X_train``: patrones de entrenamiento utilizados para clasificar un nuevo patrón. Los vecinos de un patrón dado se buscarán en este conjunto. 
*   ``y_train``: clase asociada a cada uno de los patrones del conjunto anterior.
*   ``X_test``: patrones de validación o test para los que queremos predecir su probabilidad de pertenencia a cada clase.
*   ``k``: número de vecinos más cercanos a considerar en la clasificación. Por defecto debe utilizar ``k=1``.
*   ``p``: valor de la norma para la función de distancia. Por defecto, ``p=2`` (distancia Euclídea).
*   ``clases``: lista con los nombres de las distintas clases en el conjunto de datos. Si no se proporciona, se obtiene del conjunto de entrenamiento.

Dado que para cada patrón el método debe devolver una lista de probabilidades (una por cada clase), la salida de la función será una lista de listas, una por cada instancia de test. Es decir, la salida debe seguir una estructura como la siguiente (considerando 8 clases distintas):

``
[[0.2, 0.0, 0.0, 0.0, 0.2, 0.2, 0.4, 0.0], 
[0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.4, 0.0], 
..., 
[0.2, 0.0, 0.4, 0.0, 0.2, 0.0, 0.0, 0.2]]
``

Si lo considera necesario, puede añadir más funciones auxiliares en la siguiente celda. También se considera que se utilizará la función ``distancia`` implementada anteriormente.


In [7]:
from collections import Counter

def calculate_probabilities(counter, classes, k):
  """Método que calcula la probabilidad de pertenecia a cada una de las clases

  Args:
      counter (Counter): Objeto de tipo Counter que contiene el número de veces que se repite cada clase
      classes (array): Contiene el tipo de clases que hay
      k (int): K vecinos

  Returns:
      list: Probabilidades de pertenencia a cada clase para el patrón dado
  """

  probs = []

  for cl in classes:

    probs.append(counter[cl]/k)

  return probs

def knn(X_train, y_train, X_test, k=1, p=2, clases=[]):
  '''
  Utiliza el método de los k vecinos para, a partir de los datos de entrenamiento, 
  devolver la probabilidad de pertenencia a cada clase para cada uno de los patrones
  de test.

  :param X_train: Conjunto de datos de entrenamiento. Se utiliza para buscar los k vecinos más cercanos a uno dado.
  :param y_train: Clase asociada a cada uno de los patrones de entrenamiento. Necesario para generar probabilidades.
  :param X_test: Conjunto de datos de test. Se pretende obtener la probabilidad de pertenencia a cada clase para cada uno de sus patrones.
  :param k: Número de vecinos más cercanos. Por defecto, k=1, es decir, únicamente utiliza el vecino más cercano.
  :param p: Valor de la norma para el cálculo de distancia. Por defecto, p=2, es decir, calcula distancia euclídea.
  :param clases: Distintos valores para la variable de clase. Las probabilidades calculadas seguirán el orden indicado en esta lista.
   
  :return: Lista de listas con las probabilidades de pertenencia de cada patrón de test a cada clase. Además, si no se le pasó el parámetro clases, devuelve los valores de clase.
  '''

  # Si no se proporcionan los valores para la clase 
  return_classes = False # No modificar posteriormente en el código implementado
  if len(clases) <= 0:
    return_classes = True
    clases = y_train.unique()

  # Lista con probabilidades para cada patrón. Cada elemento de la lista será otra lista.
  probabilities = []

  ###
  # COMPLETAR AQUI
  ###

  for _, test_row in X_test.iterrows():

    distances = []

    for _, train_row in X_train.iterrows():

      distances.append(distancia(test_row, train_row))

    df_distances = pd.DataFrame(data=distances, index = X_train.index, columns = ["dist"])
    
    sorted_distances = df_distances.sort_values(by=["dist"], axis = 0)

    k_distances = sorted_distances[:k]

    counter = Counter(y_train[k_distances.index])

    probabilities.append(calculate_probabilities(counter, clases, k))
    
  # Debería dejar de implementar aquí; se incluyen los return de la función
  if return_classes:
    return probabilities, clases
  else:
    return probabilities


Posteriormente, se incluye una función que será útil en el futuro. Dado un array con las probabilidades de pertenencia a cada clase para un patrón concreto, esta función se puede utilizar para devolver la clase predicha por el método, es decir, aquella con un mayor valor de probabilidad. La función recibe una lista con las probabilidades y otra con los nombres de cada una de las clases en el mismo orden que se tienen en el array de probabilidades.

Basándonos en el ejemplo de antes, si la función recibe el array ``[0.333, 0.667, 0]`` y el array ``['A', 'B', 'C']``, debe devolver 'B' como clase predicha.

Además, en caso de empate, la función escoge una clase aleatoriamente de entre aquellas con misma probabilidad. Así, nos aseguramos de que el método puede dar una salida en todos los casos.

In [8]:
import random

def clase_max_prob(probabilidades, clases, seed=None):
  '''
  Devuelve la clase predicha para un patrón a partir del vector de probabilidades asociado a cada clase.
  En caso de empate, devuelve una clase aleatoriamente de entre las que tenían
    el mayor valor de probabilidad.

  :param probabilidades: Lista con los valores de probabilidad de pertenencia a cada clase. Deben presentarse en el mismo orden de clases que el parámetro clases.
  :param clases: Distintos valores para la variable de clase. Útil para obtener la clase predicha a partir de las probabilidades.
  :param seed: Semilla para números aleatorios. Si es None (por defecto), no se asigna ninguna semilla dentro de la función.
   
  :return: Clase con vayor malor de probabilidad. Será un valor de la lista clases.
  '''
  # Asignar semilla de números aleatorios si es necesario
  if seed is not None:
    np.random.seed(seed)
  
  # Buscar mayor probabilidad 
  max_prob = max(probabilidades)

  # Buscar indice(s) de las celdas con mayor probabilidad
  indices = [index for index, p in enumerate(probabilidades) if p == max_prob]

  # Si existen varias probabilidades que coinciden en el valor mayor, escoger una aleatoria
  if len(indices) > 1:
    clase = clases[np.random.choice(indices)]
  else:
    clase = clases[indices[0]]

  # Devolver clase correspondiente con la mayor probabilidad
  return clase


def class_from_pred(probabilities, clases, seed=0):
  '''
  Devuelve una lista con las clases predichas, dada una lista de listas con las probabilidades para cada clase.
  Similar a clase_max_prob, pero recibe las probabilidades de todo el conjunto de test/validacion
  '''
  pred = []
  for pr in probabilities:
    pred.append(clase_max_prob(pr, clases, seed))

  return pred

## 3. Implementar kNN con pesos (1 punto)

Posteriormente, se busca que se implemente el método kNN donde los k patrones más cercanos no tienen el mismo peso en la decisión final, sino que su peso es inversamente proporcional al cuadrado de la distancia con el patrón de test (ver sección *Función de combinación* en la lectura obligatoria de la lección 1 de la semana 4).

En este caso, la probabilidad de salida para un patrón se calcularía en dos pasos:
1.   Calcular *peso* o *verosimilitud* de pertenencia del patrón $X$ cada clase $C_i$. Para ello, se seguirá la función siguiente, donde $Z$ sería cada vecino perteneciendo a la clase $C_i$, y $d(...)$ la función de distancia. Nótese que valores mayores de distancia entre el patrón $X$ y el vecino conllevan valores menores de *verosimilitud*; es decir, patrones más cercanos resultan en mayores valores de verosimilitud para dicha clase.
$$v_{X, C_i} = \sum_{Z \in C_i}{\frac{1}{d(X, Z)^2}}$$
2.   La probabilidad de pertenencia a cada clase $C_i$ se calcula como la verosimilitud de pertenencia a dicha clase entre la suma de todas las verosimilitudes para un patrón dado.
$$P(C_i|X) = \frac{v_{X, C_i}}{\sum_{j=1}^{\textrm{nClases}}{v_{X, C_j}}}$$

Los parámetros y salida del nuevo método ``knn_pesos`` seguirá la misma estructura que el método anterior. La diferencia principal es cómo calcular las probabilidades de pertenencia a la clase.

En la siguiente celda contiene el código a completar en este apartado. Si lo considera oportuno, puede crear las funciones auxiliares que necesite y utilizar cualquier función creada con anterioridad. Puede apoyarse en gran medida en el código implementado anteriormente.

In [9]:
import math

def calculate_probabilities_pesos(df_dis_class, classes):
  probabilities = []

  # Calculamos las verosimilitudes del patrón a todas las clases
  verosimilies = {}
  for cl in classes:

    k_distances_cl = np.array(df_dis_class[df_dis_class["class"] == cl]["dist"])

    if len(k_distances_cl) == 0:
      verosimilies[cl] = 0
    else:

      summ = 0

      for dis in k_distances_cl:

        summ += 1/dis**2

      verosimilies[cl] = summ
    
  # Calculamos la probabilidad de pertenencia del patrón a todas las clases
  summatory_ver = np.sum(list(verosimilies.values()))

  for cl in classes:

    probabilities.append(verosimilies[cl] / summatory_ver)

  return probabilities

def knn_pesos(X_train, y_train, X_test, k=1, p=2, clases=[]):
  '''
  Utiliza el método de los k vecinos para, a partir de los datos de entrenamiento, 
  devolver la probabilidad de pertenencia a cada clase para cada uno de los patrones
  de test.

  :param X_train: Conjunto de datos de entrenamiento. Se utiliza para buscar los k vecinos más cercanos a uno dado.
  :param y_train: Clase asociada a cada uno de los patrones de entrenamiento. Necesario para generar probabilidades.
  :param X_test: Conjunto de datos de test. Se pretende obtener la probabilidad de pertenencia a cada clase para cada uno de sus patrones.
  :param k: Número de vecinos más cercanos. Por defecto, k=1, es decir, únicamente utiliza el vecino más cercano.
  :param p: Valor de la norma para el cálculo de distancia. Por defecto, p=2, es decir, calcula distancia euclídea.
  :param clases: Distintos valores para la variable de clase. Las probabilidades calculadas seguirán el orden indicado en esta lista.
   
  :return: Lista de listas con las probabilidades de pertenencia de cada patrón de test a cada clase. Además, si no se le pasó el parámetro clases, devuelve los valores de clase.
  '''

  ###
  # COMPLETAR AQUI
  ###
  # Si no se proporcionan los valores para la clase 
  return_classes = False # No modificar posteriormente en el código implementado
  if len(clases) <= 0:
    return_classes = True
    clases = y_train.unique()

  # Lista con probabilidades para cada patrón. Cada elemento de la lista será otra lista.
  probabilities = []

  for _, test_row in X_test.iterrows():

    distances = []

    for _, train_row in X_train.iterrows():

      distances.append(distancia(test_row, train_row))

    df_distances = pd.DataFrame(data=distances, index = X_train.index, columns = ["dist"])
    
    sorted_distances = df_distances.sort_values(by=["dist"], axis = 0)

    k_distances = sorted_distances[:k]

    class_k_distances = y_train[k_distances.index]

    k_distances["class"] = class_k_distances

    probabilities.append(calculate_probabilities_pesos(k_distances, clases))
    
  # Debería dejar de implementar aquí; se incluyen los return de la función
  if return_classes:
    return probabilities, clases
  else:
    return probabilities


## 4. Estudio de los parámetros (0.75 puntos)

Teniendo implementados nuestros dos métodos, en este cuarto apartado vamos a hacer un estudio de los parámetros de ambos métodos.

Dado que el proceso de selección de parámetros se realiza para escoger la mejor configuración a utilizar posteriormente en los datos de test, dicho estudio se ha de realizar utilizando **solo** los datos de entrenamiento, y nunca los de test.

El objetivo de este cuarto apartado es que, tomando los datos de entrenamiento, se vuelva a hacer una partición de datos en entrenamiento y validación, de modo que se entrenen varios métodos utilizando distintos parámetros con los datos de entrenamiento, y se evalúen utilizando los de validación. Además, se espera que este proceso se realice varias veces con distintas particiones, para evitar sesgos por la elección de las particiones de entrenamiento/validación.

Debe completar la función ``validate_knn`` donde, a partir de los datos de entrenamiento y validación generados internamente debe: 1) entrenar un modelo knn (clásico o con pesos, dependiendo del parámetro ``usar_pesos``) y obtener probabilidades predichas en validación; 2) obtener la clase predicha para cada instancia de validación; y 3) calcular ciertas métricas de evaluación. 

La función ``repeat_validation_knn`` se encuentra completamente implementada. Esta función realiza varias llamadas a la función ``validate_knn`` para estimar las métricas de evaluación en distintos escenarios de particiones de entrenamiento/validación. 

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

def validate_knn(X, y, k=1, p=2, usar_pesos=False, validation_size=0.33, random_state=0):
  '''
  Esta función realiza un particionado de los datos de manera interna entre entrenamiento y validación.
  Utiliza la partición interna de entrenamiento para "entrenar" kNN y predice sobre los de test.
  Devuelve el valor de varias métricas de evaluación sobre el conjunto de validación.
  Dependiendo del parámetro usar_pesos, utilizará la versión clásica de kNN (False), o la que utiliza la distancia como peso para calcular la probabilidad (True)

  :param X: Datos de entrada a utilizar.
  :param y: Clase asociada a cada patrón de X.
  :param k: Número de vecinos más cercanos. Por defecto, k=1, es decir, únicamente utiliza el vecino más cercano.
  :param p: Valor de la norma para el cálculo de distancia. Por defecto, p=2, es decir, calcula distancia euclídea.
  :param usar_pesos: Parámetro que indica si utilizar la versión clásica de kNN (False), o si los patrones más cercanos tienen más peso en la predicción (True). Por defecto, utiliza la versión clásica (False)
  :param validation_size: Ratio de patrones que se utilizarán como partición de validación
  :param random_state: Semilla aleatoria para generar las particiones de entrenamiento/validación.

  :return: Lista con valores para distintas métricas de evaluación
  '''
  clases = y.unique()

  # Particionar los datos en puro entrenamiento, y validación
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_size, random_state=random_state)

  # Entrenar knn con datos de entrenamiento, y predecir sobre los de validación
  
  ###
  # COMPLETAR AQUI
  #   Se proporcionan como comentarios posibles partes de la función para que sirva de ayuda
  #   Sin embargo, no es necesario seguir esas recomendaciones
  ###

  if usar_pesos:
    y_proba = knn_pesos(X_train, y_train, X_val, k=k, p=p, clases=clases)
  else:
    y_proba = knn(X_train, y_train, X_val, k=k, p=p, clases=clases)

  # Para cada array en y_proba, obtener la clase predicha
  y_pred = []

  y_pred = class_from_pred(y_proba, clases, seed=0)


  # Debería dejar de implementar aquí
  # Calcular métricas de evaluación
  #   El estudiante puede incluir otras si lo considera oportuno, y modificar la función para que las devuelva
  accuracy = accuracy_score(y_val, y_pred)
  f1 = f1_score(y_val, y_pred, average='macro')
  kappa = cohen_kappa_score(y_val, y_pred)

  return accuracy, f1, kappa


def repeat_validation_knn(X, y, k=1, p=2, usar_pesos=False, validation_size=0.33, iter=10, random_state=0, verbose=True):
  '''
  Esta función hace uso de la función validate_knn para repetir el proceso varias veces,
    devolviendo el valor medio para cada una de las métricas.
  De este modo, repitiendo el proceso varias veces con distintas particiones, se busca
    reducir el sesgo producido por dicha elección de particiones de entrenamiento/validación. 

  :param X: Datos de entrada a utilizar.
  :param y: Clase asociada a cada patrón de X.
  :param k: Número de vecinos más cercanos. Por defecto, k=1, es decir, únicamente utiliza el vecino más cercano.
  :param p: Valor de la norma para el cálculo de distancia. Por defecto, p=2, es decir, calcula distancia euclídea.
  :param usar_pesos: Parámetro que indica si utilizar la versión clásica de kNN (False), o si los patrones más cercanos tienen más peso en la predicción (True). Por defecto, utiliza la versión clásica (False)
  :param validation_size: Ratio de patrones que se utilizarán como partición de validación
  :param iter: Número de iteraciones o repeticiones distintas, utilizando distintas particiones
  :param random_state: Semilla aleatoria para generar las particiones de entrenamiento/validación.
  :param verbose: Indica si muestra cierta información sobre el proceso de validación (iteraciones). Por defecto, True.

  :return: Lista con valores para distintas métricas de evaluación
  '''
  avg_accuracy = 0 
  avg_f1 = 0
  avg_kappa = 0
  for i in range(iter):
    if verbose:
      print('Iteracion ' + str(i+1) + ' de ' + str(iter))
    accuracy, f1, kappa = validate_knn(X, y, k=k, p=p, usar_pesos= usar_pesos, validation_size=validation_size, random_state = ((random_state+i)*i))
    avg_accuracy += accuracy
    avg_f1 += f1
    avg_kappa += kappa
  
  avg_accuracy = avg_accuracy / iter
  avg_f1 = avg_f1 / iter
  avg_kappa = avg_kappa / iter

  return avg_accuracy, avg_f1, avg_kappa

En la siguiente celda, debe utilizar las funciones anteriores (``repeat_validation_knn``) para probar varias combinaciones de parámetros para ambos métodos de knn implementados, y determinar, razonadamente, cuál es la mejor combinación de parámetros en cada caso. Incluya el código necesario en la siguiente celda, y razone su respuesta al final de esta misma celda.

Notas:
*   Puede probar, por ejemplo, ``p=[1, 2]``, y ``k=[1, 3, 5, 9]``, para ambos métodos. Sin embargo, el estudiante puede escoger otras combinaciones si así lo desea.
*   En cada caso, idealmente se realizarían 10 iteraciones (parámetro ``iter=10``). Sin embargo, si el proceso es muy costoso, podría reducirlo a 5 o 3 iteraciones unicamente.
*   Todas las llamadas a la función ``repeat_validation_knn`` deben hacerse con el mismo valor para el parámetro ``random_state``, asegurándonos así una comparación justa sobre las mismas particiones.
*   El ratio de instancias de validación queda a elección del usuario. Por lo general, suele ser un valor entre 0.1 y 0.33; también suele estar relacionado con el número de iteraciones (si se realizan 10 iteraciones podría dejarse en 0.1; si se realizan unicamente 3 iteraciones por ejemplo, podría fijarse a 0.33). Queda a elección del estudiante.

¿Cuál es la mejor combinación de parámetros para cada modelo de knn (con y sin pesos), y por qué? ¿Qué conclusiones puede sacar a partir de los resultados obtenidos?

**RESPUESTA**: 

Tras reducir el tiempo computacional de entrenamiento de 10 a 5 iteraciones, podemos observar que el parámetro _p_ no afecta en el proceso de entrenamiento, ya que los valores de las métricas de evaluación extraidas tras el proceso de entrenamiento son iguales para p=1 y p=2. Además, también se puede observar que, para valores de K distintos de 1, el proceso de entrenamiento funciona peor que cuando el número de vecinos más cercanos es 1. Aún así, los resultados obtenidos en todas las configuraciones es bastante buenos. Aún así, se puede observar que el algoritmo de _kNN_ básico, devuelve peores resultados que el algoritmo _kNN_ con pesos para valores de K mayores que 1. En definitiva, los mejores resultados obtenidos han sido extraidos usando una de las dos siguientes configuraciones:

- _kNN_ básico con K = 1, pudiendo utlizar la fórmula de la distancia genérica o euclidiana.

- _kNN_ con pesos con K = 1, pudiendo utlizar la fórmula de la distancia genérica o euclidiana.

In [11]:
###
# COMPLETAR AQUI
###

# Incluya el código necesario, realizando llamadas a la función repeat_validation_knn
#   para cumplir con lo propuesto en la celda anterior y determinar la mejor
#   combinación de parámetros para cada método de knn

knn_val_1 = {}
knn_val_2 = {}
knn_pesos_val_1 = {}
knn_pesos_val_2 = {}

for i in [1, 3, 5, 9]:
    print("KNN: k = " + str(i) + " - " + "p = 1")
    knn_val_1["k_" + str(i)] = repeat_validation_knn(X_train, y_train, k=i, p=1, usar_pesos=False, validation_size=0.33, iter=5, random_state=0, verbose=True)

    print("KNN: k = " + str(i) + " - " + "p = 2")
    knn_val_2["k_" + str(i)] = repeat_validation_knn(X_train, y_train, k=i, p=2, usar_pesos=False, validation_size=0.33, iter=5, random_state=0, verbose=True)

    print("KNN Pesos: k = " + str(i) + " - " + "p = 1")
    knn_pesos_val_1["k_" + str(i)] = repeat_validation_knn(X_train, y_train, k=i, p=1, usar_pesos=True, validation_size=0.33, iter=5, random_state=0, verbose=True)

    print("KNN Pesos: k = " + str(i) + " - " + "p = 2")
    knn_pesos_val_2["k_" + str(i)] = repeat_validation_knn(X_train, y_train, k=i, p=2, usar_pesos=True, validation_size=0.33, iter=5, random_state=0, verbose=True)



KNN: k = 1 - p = 1
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN: k = 1 - p = 2
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN Pesos: k = 1 - p = 1
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN Pesos: k = 1 - p = 2
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN: k = 3 - p = 1
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN: k = 3 - p = 2
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN Pesos: k = 3 - p = 1
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN Pesos: k = 3 - p = 2
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN: k = 5 - p = 1
Iteracion 1 de 5
Iteracion 2 de 5
Iteracion 3 de 5
Iteracion 4 de 5
Iteracion 5 de 5
KNN: k = 5 - p = 2
Iteracion 1 de 5
Iter

In [12]:
for i in [1,3,5,9]:
    print("kNN - K="+str(i)+" - p=1 - it=5")
    print("Acc: " + str(knn_val_1["k_" + str(i)][0]))
    print("F1: " + str(knn_val_1["k_" + str(i)][1]))
    print("Kappa: " + str(knn_val_1["k_" + str(i)][2]))

kNN - K=1 - p=1 - it=5
Acc: 0.97703081232493
F1: 0.977622443956973
Kappa: 0.973686684696603
kNN - K=3 - p=1 - it=5
Acc: 0.9361344537815126
F1: 0.9367595027591425
Kappa: 0.9268665212021043
kNN - K=5 - p=1 - it=5
Acc: 0.90812324929972
F1: 0.9083411049135595
Kappa: 0.894789723208506
kNN - K=9 - p=1 - it=5
Acc: 0.8330532212885154
F1: 0.8320731643898156
Kappa: 0.8087778714779074


In [13]:
for i in [1,3,5,9]:
    print("kNN - K="+str(i)+" - p=2 - it=5")
    print("Acc: " + str(knn_val_2["k_" + str(i)][0]))
    print("F1: " + str(knn_val_2["k_" + str(i)][1]))
    print("Kappa: " + str(knn_val_2["k_" + str(i)][2]))

kNN - K=1 - p=2 - it=5
Acc: 0.97703081232493
F1: 0.977622443956973
Kappa: 0.973686684696603
kNN - K=3 - p=2 - it=5
Acc: 0.9361344537815126
F1: 0.9367595027591425
Kappa: 0.9268665212021043
kNN - K=5 - p=2 - it=5
Acc: 0.90812324929972
F1: 0.9083411049135595
Kappa: 0.894789723208506
kNN - K=9 - p=2 - it=5
Acc: 0.8330532212885154
F1: 0.8320731643898156
Kappa: 0.8087778714779074


In [14]:
for i in [1,3,5,9]:
    print("kNN Pesos - K="+str(i)+" - p=1 - it=5")
    print("Acc: " + str(knn_pesos_val_1["k_" + str(i)][0]))
    print("F1: " + str(knn_pesos_val_1["k_" + str(i)][1]))
    print("Kappa: " + str(knn_pesos_val_1["k_" + str(i)][2]))

kNN Pesos - K=1 - p=1 - it=5
Acc: 0.97703081232493
F1: 0.977622443956973
Kappa: 0.973686684696603
kNN Pesos - K=3 - p=1 - it=5
Acc: 0.9680672268907562
F1: 0.9685775791988718
Kappa: 0.9634227409091846
kNN Pesos - K=5 - p=1 - it=5
Acc: 0.9669467787114845
F1: 0.967121673025817
Kappa: 0.9621418866908652
kNN Pesos - K=9 - p=1 - it=5
Acc: 0.961344537815126
F1: 0.9613650602383237
Kappa: 0.9557193216245305


In [15]:
for i in [1,3,5,9]:
    print("kNN Pesos - K="+str(i)+" - p=2 - it=5")
    print("Acc: " + str(knn_pesos_val_2["k_" + str(i)][0]))
    print("F1: " + str(knn_pesos_val_2["k_" + str(i)][1]))
    print("Kappa: " + str(knn_pesos_val_2["k_" + str(i)][2]))

kNN Pesos - K=1 - p=2 - it=5
Acc: 0.97703081232493
F1: 0.977622443956973
Kappa: 0.973686684696603
kNN Pesos - K=3 - p=2 - it=5
Acc: 0.9680672268907562
F1: 0.9685775791988718
Kappa: 0.9634227409091846
kNN Pesos - K=5 - p=2 - it=5
Acc: 0.9669467787114845
F1: 0.967121673025817
Kappa: 0.9621418866908652
kNN Pesos - K=9 - p=2 - it=5
Acc: 0.961344537815126
F1: 0.9613650602383237
Kappa: 0.9557193216245305


## 5. Comparación con otros métodos (0.25 puntos)

Por último, tras seleccionar los mejores parámetros para ambas versiones de kNN, vamos a comparar el rendimiento de nuestro método contra otros disponibles en scikit-learn. Para ello, entrenaremos todos los modelos utilizando el mismo conjunto de entrenamiento, y evaluaremos sobre el conjunto de test (que recordemos, no se ha utilizado aún).
El objetivo de este notebook no es aprender cómo generar otros modelos de clasificación o analizar su funcionamiento, sino que simplemente vamos a obtener algunas métricas de evaluación de los mismos.

En primer lugar, debe completar las líneas donde se genera kNN sobre los datos de entrenamiento y se evalúa sobre los de test. Simplemente tiene que incluir los parámetros que consideró que obtenían un mejor resultado según el apartado anterior.

Tras completar la siguiente celda y ejecutar ambas, responda a las siguientes preguntas, en esta misma celda de texto:
*   En los experimentos anteriores, qué knn era mejor, ¿con o sin pesos? ¿Y sobre los datos de test? ¿En ambos casos coincide que es el mismo método el mejor o no? ¿Y por qué crees que ocurre?. **RESPUESTA**: El mejor valor de K observado es de 1 vecino más cercano, arrojando los mismos resultados ambos métodos (con y sin pesos). Aún así, para valores mayores de K, se puede observar mejores resultados usando el método kNN con pesos. Aún así, sería interesante poder aumentar el número de iteraciones en el proceso de entrenamiento para poder observar si se produce un sobreaprendizaje en alguno de los dos métodos.
*   De todos los métodos evaluados sobre datos de test, ¿cuál obtiene mejores resultados? **RESPUESTA**: Se puede observar que los mejores resultados han sido obtenidos usando los algoritmos de kNN con y sin pesos, seguidos del método de análisis discriminante lineal.



In [16]:
# Entrenamiento y evaluación de kNN, utilizando datos de test para evaluar

from sklearn.metrics import *

# Funcion para calcular métricas de interés, e imprimirlas por pantalla
def evaluar_e_imprimir(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  print('   acc: ' + str(accuracy))
  f1 = f1_score(y_test, y_pred, average='macro')
  print('   f1: ' + str(f1))
  kappa = cohen_kappa_score(y_test, y_pred)
  print('   kappa: ' + str(kappa))

# Valores de clases
clases = y.unique()

# Evaluar knn básico 
# COMPLETAR los ... con valores de correspondientes
knn_proba = knn(X_train, y_train, X_test, k=1, p=1, clases=clases)
knn_pred = class_from_pred(knn_proba, clases, seed=0)
print('kNN básico')
evaluar_e_imprimir(y_test, knn_pred)

# Evaluar knn básico 
# COMPLETAR con valores de correspondientes
knn_pesos_proba = knn_pesos(X_train, y_train, X_test, k=1, p=2, clases=clases)
knn_pesos_pred = class_from_pred(knn_pesos_proba, clases, seed=0)
print('\nkNN con pesos')
evaluar_e_imprimir(y_test, knn_pesos_pred)

kNN básico
   acc: 0.9876543209876543
   f1: 0.9878905078998267
   kappa: 0.9858644910780507

kNN con pesos
   acc: 0.9876543209876543
   f1: 0.9878905078998267
   kappa: 0.9858644910780507


In [17]:
# Entrenamiento de otros métodos (simplemente ejecutar y analizar resultados)

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb_pred = GaussianNB().fit(X_train, y_train).predict(X_test)
print('\nNaïve Bayes')
evaluar_e_imprimir(y_test, nb_pred)

# Árbol de decisión
from sklearn.tree import DecisionTreeClassifier
dt_pred = DecisionTreeClassifier(random_state=0).fit(X_train, y_train).predict(X_test)
print('\nÁrbol de decisión')
evaluar_e_imprimir(y_test, dt_pred)

# Análisis discriminante lineal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_pred = LinearDiscriminantAnalysis().fit(X_train, y_train).predict(X_test)
print('\nAnálisis discriminante lineal')
evaluar_e_imprimir(y_test, lda_pred)

# Regresión logística
from sklearn.linear_model import LogisticRegression
lr_pred = LogisticRegression(random_state=0, max_iter=1000, multi_class='multinomial').fit(X_train, y_train).predict(X_test)
print('\nRegresión logística')
evaluar_e_imprimir(y_test, lr_pred)


Naïve Bayes
   acc: 0.7407407407407407
   f1: 0.7368864398944323
   kappa: 0.7025346201348737

Árbol de decisión
   acc: 0.8395061728395061
   f1: 0.8395742690685304
   kappa: 0.8163605646084255

Análisis discriminante lineal
   acc: 0.9660493827160493
   f1: 0.9671276111635012
   kappa: 0.9611531963594746

Regresión logística
   acc: 0.9012345679012346
   f1: 0.9016944912867919
   kappa: 0.8869972752043597
